In [1]:
import sqlalchemy as db
import os
import pandas as pd
pd.set_option('display.max_rows', 10)

In [2]:
dbURL = 'postgres+psycopg2://maxcarey:<password>@localhost:5432/corpus'
engine = db.create_engine(dbURL)

# Database specs
For me it would be easier to visualize these SQL commands if there was some kind of diagram, that is why I mention this as a potential task.

- Corpus
    - corpus_id
    - source_html_id
    - word_id
- Lexicon Table
    - id
    - word
    - lemma
    - pos
- Sources
    - source_html_id
    - text
- Source Info
    - source_html_id
    - num_words
    - genre
    - country
    - website
    - url
    - title

# 1. Get total words by country in corpus
I wanted to get total words by country because I need to normalize the frequencies depending on how representative the sampels of each countrty are insode of the corpus. Later in the script, you can see that I use the object `word_count_by_country` and do some math in pandas (not raw SQL) to get the normalized frequency. Perhaps this can all be built in to the final SQL queries.

In [3]:
query = '''
SELECT COUNT(*), country
FROM corpus
LEFT JOIN source_info
ON source_info.source_html_id = corpus.source_html_id
GROUP BY country
ORDER BY count DESC
'''
word_count_by_country = pd.read_sql(query, engine)

In [4]:
word_count_by_country

,count,country
0,528258,ES
1,286231,MX
2,208283,US
3,199201,AR
4,174540,CO
...,...,...
16,39235,BO
17,38226,SV
18,32021,PA
19,27679,CR


# 2. Get all the bigrams bigrams estar padre, where estar is a lemma

From the results, of this section. I think my SQL query is completely off, I thought this was giving me all of the n-grams of lemma-estar + word-padre, as well as the country and the soruce_html id of where these n-grams ocurred. But if you search the actual text sources (source.txt in this directory) you cannot find any strings of "estuvo padre"

In [5]:
# Get queries where the words are
query = '''
        SELECT source_info.country, lex1.word, lex2.word, source_info.source_html_id
        FROM source_info, lexicon as lex1, lexicon as lex2, corpus as corpus1, corpus as corpus2
        WHERE lex1.lemma = 'estar'
        AND lex2.word = 'padre'
        AND lex1.id = corpus1.word_id
        AND lex2.id = corpus2.word_id
        AND corpus1.corpus_id::BIGINT = corpus2.corpus_id::BIGINT + 1
        GROUP BY lex1.word, lex2.word, source_info.country, source_info.source_html_id
        ORDER BY count(source_info.country) DESC
    '''
    
    # Merge in query and engine

pd.read_sql(query, engine)

,country,word,word,source_html_id
0,PE,estuvo,padre,1000124
1,PE,estuvo,padre,1001124
2,AR,estuvo,padre,100124
3,PE,estuvo,padre,1002124
4,PE,estuvo,padre,1003124
...,...,...,...,...
8407,PE,estaba,padre,995124
8408,PE,estaba,padre,996124
8409,PE,estaba,padre,997124
8410,PE,estaba,padre,998124


## Problem
This code shows the problem, you would think from the output above
that the string "estuvo padre" would be in the tale

In [6]:
query = '''
    SELECT text
    FROM source
    WHERE source_html_id='1000124'
'''
first_example = pd.read_sql(query, engine)

In [7]:
text_as_string = first_example.iloc[0,0]

In [8]:
text_as_string[0:300]

'Cinco acciones para incrementar la seguridad en Firefox Con los años , Firefox se ha convertido para muchos en uno de los navegadores web actuales más seguros . Pero , como ocurre con otros muchos browsers , el nivel de seguridad depende de su correcta configuración . Algunas de las características '

In [9]:
"estuvo padre" in text_as_string.lower()

False

# 3. Count the bigrams from the previous step by country
If step 2 worked, then this would be the way to do a count by each country

In [10]:
# Get absolute counts of bigrams by country
query = '''
        SELECT COUNT(source_info.country), source_info.country, lex1.word, lex2.word
        FROM source_info, lexicon as lex1, lexicon as lex2, corpus as corpus1, corpus as corpus2
        WHERE lex1.lemma = 'estar'
        AND lex2.word = 'padre'
        AND lex1.id = corpus1.word_id
        AND lex2.id = corpus2.word_id
        AND corpus1.corpus_id::BIGINT = corpus2.corpus_id::BIGINT + 1
        GROUP BY lex1.word, lex2.word, source_info.country
        ORDER BY count(source_info.country) DESC
    '''
    
    # Merge in query and engine

pd.read_sql(query, engine)

,count,country,word,word
0,840,ES,estuvo,padre
1,582,MX,estuvo,padre
2,420,ES,estaba,padre
3,420,ES,está,padre
4,420,ES,están,padre
...,...,...,...,...
79,31,PA,están,padre
80,31,PR,estaba,padre
81,31,PA,está,padre
82,31,PA,estaba,padre


# 4. Get bigrams
Finally, this would calculate the absolute frquency doing some pandas math from step 1.

In [11]:
def get_bigrams():

    query = '''
        SELECT SUM(counts.count), counts.country
        FROM (SELECT COUNT(source_info.country), source_info.country, lex1.word, lex2.word
        FROM source_info, lexicon as lex1, lexicon as lex2, corpus as corpus1, corpus as corpus2
        WHERE lex1.lemma = 'estar'
        AND lex2.word = 'padre'
        AND lex1.id = corpus1.word_id
        AND lex2.id = corpus2.word_id
        AND corpus1.corpus_id::BIGINT = corpus2.corpus_id::BIGINT + 1
        GROUP BY lex1.word, lex2.word, source_info.country
        ORDER BY count(source_info.country) DESC) AS counts
        GROUP BY counts.country
        ORDER BY sum DESC 
    '''
    
    # Merge in query and engine
    absolute_frequency_by_country = pd.read_sql(query, engine)

    # Merge in the counts for each corpus
    absolute_frequency_by_country_counts = pd.merge(absolute_frequency_by_country,word_count_by_country,on='country',how='left')

    absolute_frequency_by_country_counts['relativeFrequency'] = (absolute_frequency_by_country_counts['sum']*10000)/absolute_frequency_by_country_counts['count']
    
    return absolute_frequency_by_country_counts
    

In [12]:
get_bigrams()

,sum,country,count,relativeFrequency
0,2100.0,ES,528258,39.753302
1,1455.0,MX,286231,50.833068
2,920.0,CO,174540,52.709981
3,905.0,AR,199201,45.431499
4,820.0,US,208283,39.369512
...,...,...,...,...
16,180.0,UY,84743,21.240692
17,175.0,PY,50201,34.859863
18,175.0,CR,27679,63.224827
19,155.0,PA,32021,48.405734


# 5. Get examples
This is a function to get examples sentences from for the string search for in steps 2-4. This is the last step once I get the the queries working

In [13]:
def get_examples():
    
    # For now this is hardcoded
    # TODO: Use string interpolation to pass in the words
    # Refactor the two functions so that they can handle unigrams, and trigrams, 4 grams, n+ grams
    # Refactor the code so if can also handle different parts of speech
    
    query = '''
    SELECT concordances.source_html_id, source.text, source_info.country
    FROM (SELECT lex1.word, lex2.word, corpus.source_html_id
    FROM lexicon as lex1, lexicon as lex2, corpus as corpus1, corpus as corpus
    WHERE lex1.lemma = 'estar'
    AND lex1.id = corpus1.word_id
    AND lex2.id = corpus.word_id
    AND corpus.corpus_id::BIGINT = corpus1.corpus_id::BIGINT + 1
    GROUP BY lex1.word, lex2.word, corpus.source_html_id) as concordances
    LEFT JOIN source
    ON concordances.source_html_id = source.source_html_id
    LEFT JOIN source_info
    ON concordances.source_html_id = source_info.source_html_id
    LIMIT 100
    '''
    
    return pd.read_sql(query, engine)

In [14]:
get_examples()

,source_html_id,text,country
0,1330124,ESTABA VIVA AL SER ARROJADA AL CONTAINER Tamañ...,AR
1,1234124,VENEZUELA SIGUE DAMNIFICADA ¿ O es que alguna ...,VE
2,1936124,Hay un hecho que no se ha publicitado mucho pe...,HN
3,607124,"Páginas Páginas vistas en total miércoles , 24...",ES
4,1081124,Enlaces Paraguay vs España cuartos de final En...,PY
...,...,...,...
95,1238124,Un millón 500 mil bolívares invirtió Gobernaci...,VE
96,260124,¡ Bienvenidos sean todos a este humilde rincón...,CO
97,386124,Seguidores Contador de visitas Solicitó a el p...,DO
98,438124,Voy a ser claro y contundente desde el princip...,ES
